# Introduction

I'm not doing anything terribly special here. Just creating a catboost classification model and soe visual representations of the results. I know that there's more work to do, but I kind of don't know where I should continue from what I have. Feedback is welcome.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/fetal-health-classification/fetal_health.csv')
data.head()

# EDA
There isn't much cleaning to do here; There aren't any NaN values to replace, and all of the data is already in numerical values. Like the task for the dataset says, there is a class imbalance, but I ignored that for now, thinking that I would come back and stratify later if it was an issue.

In [ ]:
data.fetal_health.value_counts()

In [ ]:
data.isnull().sum()

# Column Descriptions
* Baseline value: Baseline Fetal Heart Rate (FHR)
* accelerations: Number of Accelerations per second
* fetal_movement: Number of fetal movements per second
* uterine_contractions: Number of uterine contractions per second
* light_decelerations: Number of LDs per second
* severe_decelerations: Number of SDs per second
* prolonged_decelerations: Number of PDs per second
* abnormal_short_term_variability: Percentage of time with abnormal short term variability
* mean_value_of_short_term_variability: Mean value of short term variability
* percentage_of_time_with_abnormal_long_term_variability: Percentage of time with abnormal long term variability
* mean_value_of_long_term_viability: Mean value of long term variability
* histogram_width: Width of the histogram made using all values from a record
* histogram_min: Histogram minimum value
* histogram_max: Histogram maximum value
* histogram_number_of_peaks: Number of peaks in the exam histogram
* histogram_number_of_zeroes: Number of Zeroes in the exact histogram
* histogram_mode: Hist mode
* histogram_mean: Hist mean
* histogram_median: Hist median
* histogram_variance: Hist variance
* histogram_tendency: Histogram trend
* fetal_health 1=Normal, 2=Suspect, 3=Pathological

I also wanted the results to be in string form, not integer form so that I wouldn't have to remember what they mean.

In [ ]:
health_map = {
    1: 'Normal',
    2: 'Suspect',
    3: 'Pathological'
}

for i in data.index:
    data.loc[i,'fetal_health'] = health_map[data.loc[i, 'fetal_health']]
data.head(20)

# CatBoostCalassifier

I used the hyperparameters from a different notebook for a different dataset. I was also unclear on what the roc_auc meant quantitatively, so I plotted the confusion matrix by prediction to help me make sense of it.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split as TTS

X = data.drop(columns = 'fetal_health')
y = data.fetal_health

Xtrain, Xtest, ytrain, ytest = TTS(X,y, test_size = .3, random_state = 2601744, stratify = y, shuffle = True)
#Used the recommended test size of %30

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(learning_rate = 0.03, l2_leaf_reg = 1,
                        iterations = 500, depth = 9,
                        border_count = 20, eval_metric = 'AUC')

cat = cat.fit(Xtrain, ytrain,
             eval_set = (Xtest, ytest),
             early_stopping_rounds = 70, verbose = 20)

In [ ]:
from sklearn.metrics import plot_confusion_matrix as PCM

PCM(cat, Xtest, ytest, labels = ['Pathological', 'Suspect', 'Normal'], normalize = 'pred',
   cmap = 'Greens', include_values = True, xticks_rotation = 30)
plt.title('Confusion Matrix by Prediciton', fontdict = {'fontsize': 18}, pad = 15)

Here, I'm not exactly sure of where to continue, partially due to a lack of contextual knowledge and partially due to lack of experience.

On the one hand, I don't know how good this test is in the context of medical tests in general. I do know that the worst possible outcome for this test is the scenario in the top right cell: the false negative prediction of normal condition. That is the one most likely to lead to the death of a baby. Is about a %1.3 chance of a false positive normal (but acutally Pathological) acceptably low in the context of such a test? Is the false positive Normal in the case that the result should be suspect also disasterous? I don't believe that the dataset gave much of an explanation as to what the difference is between a Pathological case and a Suspect.

On the other hand, I'm not sure about what I should do first to try to improve this model. I did basically no feature engineering, but I'm not sure what I should start with to improve the model and if the results I'm seeing here indicate that I should do something first. Should I stratify the sampling first or work on tuning the hyperparameters?

Thanks for reading and any input is appreciated.